![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [36]:
# Start your code here!
import pandas as pd
ds_jobs_clean = pd.read_csv("customer_train.csv")
ds_jobs_clean.head(20)


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0


In [37]:
import pandas as pd
ds_jobs_clean = pd.read_csv("customer_train.csv")

# Convert integer columns into int32
ds_jobs_clean[["student_id", "training_hours", "job_change"]] = ds_jobs_clean[["student_id", "training_hours", "job_change"]].astype("int32")

# Convert float column into float16
ds_jobs_clean["city_development_index"] = ds_jobs_clean["city_development_index"].astype("float16")

# Convert nominal columns into categories
ds_jobs_clean[['city', 'gender', 'major_discipline', 'company_type']] = ds_jobs_clean[['city', 'gender', 'major_discipline', 'company_type']].astype("category")

# Convert relevant_experience into an ordered category
rel_cats = pd.CategoricalDtype(['No relevant experience', 'Has relevant experience'], ordered=True)

ds_jobs_clean['relevant_experience'] = ds_jobs_clean['relevant_experience'].astype(rel_cats)

# Convert enrolled_university into an ordered category
uni_cats = pd.CategoricalDtype(['no_enrollment', 'Part time course', 'Full time course'], ordered=True)
ds_jobs_clean['enrolled_university'] = ds_jobs_clean['enrolled_university'].astype(uni_cats)

# Convert education_level into an ordered category
edu_cats = pd.CategoricalDtype(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].astype(edu_cats)

# Convert experience into an ordered category
exp_strings_lst = ['<1'] + list(map(str, range(1, 21))) + ['>20']
exp_cats = pd.CategoricalDtype(exp_strings_lst, ordered=True)
ds_jobs_clean['experience'] = ds_jobs_clean['experience'].astype(exp_cats)

# Convert company_size into an ordered category
cmp_cats = pd.CategoricalDtype(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)
ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].astype(cmp_cats)

# Convert last_new_job into an ordered category
job_cats = pd.CategoricalDtype(['never', '1', '2', '3', '4', '>4'], ordered=True)
ds_jobs_clean['last_new_job'] = ds_jobs_clean['last_new_job'].astype(job_cats)

# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]
ds_jobs_clean.head(100)

print(job_cats)
print(ds_jobs_clean.dtypes)
print(ds_jobs_clean)

category
student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience       category
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                   int32
dtype: object
       student_id      city  ...  training_hours job_change
9             699  city_103  ...             123          0
12          25619   city_61  ...              23          0
31          22293  city_103  ...             141          0
34          26494   city_16  ...             145          0
40           2547  city_114  ...              14          0
...           ...       ...  ...             ...        ...
19097       25447  city_103  ...              57          0
19101  